In [1]:
def deco(func):
  def inner():
    print('running inner()')
  return inner

@deco
def target():
  print('running target()')

target()

running inner()


In [3]:
def logger(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
    result = func(*args, **kwargs)
    print(f"[LOG] {func.__name__} returned {result}")
    return result
  return wrapper

@logger
def add(a, b):
  return a + b

add(5,12)

[LOG] add called with args=(5, 12), kwargs={}
[LOG] add returned 17


17

In [4]:
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7a34279aa020>)
running register(<function f2 at 0x7a34279aaf20>)
running main()
registry -> [<function f1 at 0x7a34279aa020>, <function f2 at 0x7a34279aaf20>]
running f1()
running f2()
running f3()


In [5]:
promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * .1
  return discount

@promotion
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if item.quantity >= 10:
    return order.total() * .07
  return 0

def best_promo(order):
  return max(promo(order) for promo in promos)

In [7]:
def f1(a):
  print(a)
  print(b)

b=6
f1(3)

3
6


In [8]:
b=6
def f2(a):
  print(a)
  print(b)
  b=9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [10]:
b=6
def f3(a):
  global b
  print(a)
  print(b)
  b=9

print(f3(3))
print(b)
print(f3(3))
b=30
print(f3(3))

3
6
None
9
3
9
None
3
30
None


In [12]:
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [13]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)

  return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [14]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

('new_value', 'total')
('series',)
(<cell at 0x7a3428041030: list object at 0x7a34279f3640>,)
[10, 11, 12]


In [16]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total/count

  return averager

avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [18]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total/count

  return averager

avg = make_averager()
avg(10)

10.0

In [19]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [20]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n<2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12325785s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000057s] factorial(1) -> 1
[0.00001393s] factorial(2) -> 2
[0.00002025s] factorial(3) -> 6
[0.00002643s] factorial(4) -> 24
[0.00003264s] factorial(5) -> 120
[0.00004084s] factorial(6) -> 720
6! = 720


In [23]:
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n<2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

print(factorial.__name__)

**************************************** Calling snooze(.123)
[0.12316012s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00002241s] factorial(2) -> 2
[0.00003314s] factorial(3) -> 6
[0.00004029s] factorial(4) -> 24
[0.00004768s] factorial(5) -> 120
[0.00005960s] factorial(6) -> 720
6! = 720
factorial
